In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

Mounted at /content/drive


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import pandas as pd

file_path = '/content/drive/My Drive/TCC - UFMS/TCC - UFMS - final/Corpus/merged_df.csv'
df = pd.read_csv(file_path)

In [4]:
df.head()

,Nome da Música,Artista,Gênero Musical,Letra da Música
0,Carolina,Seu Jorge,MPB,Carolina é uma menina bem difícil de esquecer ...
1,Epitáfio,Titãs,Rock,Devia ter amado mais Ter chorado mais Ter vist...
2,Lugar Ao Sol,Charlie Brown Jr.,Rock,"Que bom viver, como é bom sonhar E o que ficou..."
3,Relicário - Ao Vivo,Cássia Eller,Rock,É uma índia com colar A tarde linda que não qu...
4,Você Me Vira A Cabeça (Me Tira Do Sério),Alcione,Samba,"Você me vira a cabeça, me tira do sério Destró..."


In [5]:
import spacy

!python -m spacy download pt_core_news_lg
pln = spacy.load('pt_core_news_lg')
pln

from spacy.matcher import DependencyMatcher

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 600.1 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [6]:
sujeitos_infantis = ["criança", "adolescente", "mocinha", "mocinho","garotinha","garotinho", "ninfetinha", "novinha", "novinho", "ninfeta", "bebê", "neném", "pivete", "anjinho", "pirralha", "pirralho", "lolita", "bonequinha", "bebezinha", "bebezinho", "moleca", "nenenzinha", "nenenzinho", "menininha", "menininho"]

In [7]:
dep_matcher = DependencyMatcher(vocab=pln.vocab)

vozpassiva = [{"RIGHT_ID": "verbo", "RIGHT_ATTRS": {"MORPH": {"IN": ["Gender=Fem|Number=Sing|VerbForm=Part|Voice=Pass", "Gender=Fem|Number=Plur|VerbForm=Part|Voice=Pass"]}}},
              {"LEFT_ID": "verbo", "REL_OP": ">", "RIGHT_ID": "sujeito", "RIGHT_ATTRS": {"DEP": "nsubj:pass", "LOWER": {"IN": sujeitos_infantis}}},
              {"LEFT_ID": "verbo", "REL_OP": ">", "RIGHT_ID": "auxliar", "RIGHT_ATTRS": {"DEP": "aux:pass", "POS":"AUX"}}]

sujeitoauxadjetivo = [{"RIGHT_ID": "adjetivo", "RIGHT_ATTRS": {"POS": "ADJ"}},
                      {"LEFT_ID": "adjetivo", "REL_OP": ">", "RIGHT_ID": "sujeito", "RIGHT_ATTRS": {"DEP": "nsubj", "LOWER": {"IN": sujeitos_infantis}}},
                      {"LEFT_ID": "adjetivo", "REL_OP": ">", "RIGHT_ID": "auxiliar", "RIGHT_ATTRS": {"POS":{"IN": ["AUX", "COP"]}}}]

ficar_parecer_tornar = [{"RIGHT_ID": "verbo", "RIGHT_ATTRS": {"LEMMA":
                          {"IN": ["parecer", "ficar", "tornar"]}}},
                        {"LEFT_ID": "verbo", "REL_OP": ">", "RIGHT_ID": "sujeito", "RIGHT_ATTRS": {"DEP": "nsubj", "LOWER": {"IN": sujeitos_infantis}}},
                        {"LEFT_ID": "verbo", "REL_OP":">", "RIGHT_ID": "adjetivo", "RIGHT_ATTRS": {"DEP":"xcomp", "POS": "ADJ"}}]

sujeitoadjetivo = [{"RIGHT_ID": "noun", "RIGHT_ATTRS": {"LOWER": {"IN": sujeitos_infantis}}},
                   {"LEFT_ID": "noun", "REL_OP":">", "RIGHT_ID": "adjetivo", "RIGHT_ATTRS": {"DEP":"amod", "POS": "ADJ"}}]

substantivo = [{"RIGHT_ID": "substantivo", "RIGHT_ATTRS": {"POS": "NOUN"}},
               {"LEFT_ID": "substantivo", "REL_OP": ">", "RIGHT_ID": "sujeito", "RIGHT_ATTRS": {"DEP": "nsubj", "LOWER": {"IN": sujeitos_infantis}}},
               {"LEFT_ID": "substantivo", "REL_OP": ">", "RIGHT_ID": "auxiliar", "RIGHT_ATTRS": {"POS":{"IN":["AUX", "COP"]}}}]


dep_matcher.add('vozpassiva', patterns=[vozpassiva])
dep_matcher.add('ficar_parecer_tornar', patterns=[ficar_parecer_tornar])
dep_matcher.add('sujeitoauxadjetivo', patterns=[sujeitoauxadjetivo])
dep_matcher.add('sujeitoadjetivo', patterns=[sujeitoadjetivo])
dep_matcher.add('substantivo', patterns=[substantivo])

In [8]:
resultados = []

for index, row in df.iterrows():
    text = str(row['Letra da Música']).replace('\n', '. ')  # Substitui quebras de linha por ponto final
    doc = pln(text)
    dep_matches = dep_matcher(doc)
    for match in dep_matches:
        pattern_name = match[0]
        matches = match[1]
        if pln.vocab[pattern_name].text in ['vozpassiva', 'sujeitoauxadjetivo', 'substantivo'] :
            inicio, final = matches[1], matches[0]
            frase = doc[inicio:final+1].text
            print('Padrão:', pln.vocab[pattern_name].text, 'Frase:', frase)
            resultados.append({'Padrão': pln.vocab[pattern_name].text,
                           'Frase': frase,
                           'Nome da Música': row['Nome da Música'],
                           'Nome do Artista': row['Artista'],
                           'Gênero Musical': row['Gênero Musical'],
                           'Letra da Música': text})
        elif pln.vocab[pattern_name].text in ['ficar_parecer_tornar']:
            inicio, final = matches[1], matches[2]
            frase = doc[inicio:final+1].text
            print('Padrão:', pln.vocab[pattern_name].text, 'Frase:', frase)
            resultados.append({'Padrão': pln.vocab[pattern_name].text,
                           'Frase': frase,
                           'Nome da Música': row['Nome da Música'],
                           'Nome do Artista': row['Artista'],
                           'Gênero Musical': row['Gênero Musical'],
                           'Letra da Música': text})
        elif pln.vocab[pattern_name].text == 'sujeitoadjetivo':
            inicio, final = matches[0], matches[1]
            frase = doc[inicio:final+1].text
            print('Padrão:', pln.vocab[pattern_name].text, 'Frase:', frase)
            resultados.append({'Padrão': pln.vocab[pattern_name].text,
                           'Frase': frase,
                           'Nome da Música': row['Nome da Música'],
                           'Nome do Artista': row['Artista'],
                           'Gênero Musical': row['Gênero Musical'],
                           'Letra da Música': text})

resultados_df = pd.DataFrame(resultados)
resultados_df.head()

Padrão: sujeitoauxadjetivo Frase: bebê cara é foda
Padrão: sujeitoauxadjetivo Frase: bebê cara é foda
Padrão: sujeitoauxadjetivo Frase: bebê cara é foda
Padrão: sujeitoauxadjetivo Frase: bebê cara é foda
Padrão: sujeitoadjetivo Frase: 
Padrão: sujeitoadjetivo Frase: criança má
Padrão: substantivo Frase: 
Padrão: substantivo Frase: 
Padrão: substantivo Frase: 
Padrão: substantivo Frase: 
Padrão: substantivo Frase: 
Padrão: substantivo Frase: 
Padrão: substantivo Frase: 
Padrão: substantivo Frase: 
Padrão: substantivo Frase: 
Padrão: substantivo Frase: 
Padrão: sujeitoadjetivo Frase: criança Cheia
Padrão: sujeitoadjetivo Frase: criança Cheia
Padrão: ficar_parecer_tornar Frase: novinha saliente Fica loucona
Padrão: sujeitoadjetivo Frase: novinha saliente
Padrão: sujeitoadjetivo Frase: novinha louca
Padrão: sujeitoadjetivo Frase: novinha louca
Padrão: sujeitoadjetivo Frase: ninfetinha dessa vai ser difícil
Padrão: sujeitoadjetivo Frase: ninfetinha dessa vai ser difícil
Padrão: ficar_parece

,Padrão,Frase,Nome da Música,Nome do Artista,Gênero Musical,Letra da Música
0,sujeitoauxadjetivo,bebê cara é foda,Coração de Gelo,WIU,Rap,"Get-get it, get it up, get it up, get it up, b..."
1,sujeitoauxadjetivo,bebê cara é foda,Coração de Gelo,WIU,Rap,"Get-get it, get it up, get it up, get it up, b..."
2,sujeitoauxadjetivo,bebê cara é foda,Coração de Gelo,WIU,Rap,"Get-get it, get it up, get it up, get it up, b..."
3,sujeitoauxadjetivo,bebê cara é foda,Coração de Gelo,WIU,Rap,"Get-get it, get it up, get it up, get it up, b..."
4,sujeitoadjetivo,,Roça Em Mim,Zé Felipe,Sertanejo,Coloca o capacete que lá vem pedrada E pra voc...


In [9]:
from google.colab import files
resultados_df.to_csv('resultados_df_infantil.csv', index=False)
files.download('resultados_df_infantil.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
resultados_df['Frase'] = resultados_df['Frase'].astype(str)
resultados_df['Frase'] = resultados_df['Frase'].str.strip()
resultados_df['Frase'] = resultados_df['Frase'].str.lower()
resultados_df['Frase'] = resultados_df['Frase'].str.replace(r'\s+', ' ', regex=True)
resultados_df = resultados_df.drop_duplicates(subset=['Frase', 'Nome da Música', 'Nome do Artista', 'Gênero Musical', 'Letra da Música'])
resultados_df.head()

,Padrão,Frase,Nome da Música,Nome do Artista,Gênero Musical,Letra da Música
0,sujeitoauxadjetivo,bebê cara é foda,Coração de Gelo,WIU,Rap,"Get-get it, get it up, get it up, get it up, b..."
4,sujeitoadjetivo,,Roça Em Mim,Zé Felipe,Sertanejo,Coloca o capacete que lá vem pedrada E pra voc...
5,sujeitoadjetivo,criança má,Bronzes e Cristais,Maysa,MPB,Se alguém contar pelos dedos Quantas alegrias ...
6,substantivo,,Tropicália,Caetano Veloso,MPB,Quando Pero Vaz Caminha Descobriu que as terra...
16,sujeitoadjetivo,criança cheia,Sou Rebelde (Soy Rebelde),Lilian,Jovem Guarda,Eu sou rebelde porque o mundo quis assim Porqu...


In [11]:
resultados_df = resultados_df[resultados_df['Frase'].str.strip() != ""]
resultados_df.head()

,Padrão,Frase,Nome da Música,Nome do Artista,Gênero Musical,Letra da Música
0,sujeitoauxadjetivo,bebê cara é foda,Coração de Gelo,WIU,Rap,"Get-get it, get it up, get it up, get it up, b..."
5,sujeitoadjetivo,criança má,Bronzes e Cristais,Maysa,MPB,Se alguém contar pelos dedos Quantas alegrias ...
16,sujeitoadjetivo,criança cheia,Sou Rebelde (Soy Rebelde),Lilian,Jovem Guarda,Eu sou rebelde porque o mundo quis assim Porqu...
18,ficar_parecer_tornar,novinha saliente fica loucona,Bum Bum Tam Tam,MC Fioti,Funk,É a flauta envolvente que mexe com a mente De ...
19,sujeitoadjetivo,novinha saliente,Bum Bum Tam Tam,MC Fioti,Funk,É a flauta envolvente que mexe com a mente De ...


In [12]:
resultados_df.shape

(593, 6)

In [13]:
from google.colab import files
resultados_df.to_csv('resultados_df_infantil_sem duplicata.csv', index=False)
files.download('resultados_df_infantil_sem duplicata.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
from collections import Counter

adjetivos = []
for frase in resultados_df['Frase']:
    doc = pln(frase)
    adjetivos += [token.text for token in doc if token.pos_ == "ADJ"]

contador_adjetivos = Counter(adjetivos)

ranking_adjetivos = pd.DataFrame(contador_adjetivos.items(), columns=['Adjetivo', 'Frequência'])
ranking_adjetivos = ranking_adjetivos.sort_values(by='Frequência', ascending=False).reset_index(drop=True)

print(ranking_adjetivos.head(10))

   Adjetivo  Frequência
0     feliz          23
1     louca          22
2    bonita          18
3  inocente          17
4      nova          13
5     linda          12
6  indefesa           8
7     cheia           7
8  saliente           6
9    bonito           6


In [16]:
print(ranking_adjetivos.head(20))

       Adjetivo  Frequência
0         feliz          23
1         louca          22
2        bonita          18
3      inocente          17
4          nova          13
5         linda          12
6      indefesa           8
7         cheia           7
8      saliente           6
9        bonito           6
10        livre           6
11        pobre           6
12      pequena           6
13       grande           5
14   terrorista           5
15       triste           5
16   experiente           5
17  inteligente           5
18      novinha           5
19          bom           5


In [15]:
from google.colab import files
resultados_df.to_csv('resultados_df_infantil_sem_duplicata.csv', index=False)
files.download('resultados_df_infantil_sem_duplicata.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>